In [1]:
import pandas as pd

In [2]:
all_cols_df = pd.read_csv('game_data/game_stats.csv')

In [3]:
import numpy as np
df = all_cols_df.iloc[:, 0:12]
df['Result'] = np.where(df['HomeScore'] > df['AwayScore'], 1, -1)


df.head()

,Date,KickOff,HomeTeam,AwayTeam,Venue,HomeScore,AwayScore,PlayOff Game?,HomeGoals,HomeBehinds,AwayGoals,AwayBehinds,Result
0,25-Apr-18,15:20,Collingwood,Essendon,MCG,101,52,NaN,14,17,7,10,1
1,24-Apr-18,19:25,Melbourne,Richmond,MCG,56,102,NaN,8,8,15,12,-1
2,22-Apr-18,16:40,Brisbane,Gold Coast,Gabba,71,76,NaN,10,11,11,10,-1
3,22-Apr-18,15:20,North Melbourne,Hawthorn,Etihad Stadium,98,70,NaN,14,14,11,4,1
4,21-Apr-18,18:10,Fremantle,Western Bulldogs,Simonds Stadium,108,54,NaN,16,12,8,6,1


In [4]:
import datetime as dt
teams = ['Collingwood', 'Essendon', 'Brisbane', 'Gold Coast', 'North Melbourne', 'Hawthorn', 'Fremantle', 'Western Bulldogs', 'Port Adelaide', 'Geelong', 'Carlton', 'West Coast', 'St Kilda', 'GWS Giants', 'Sydney', 'Adelaide', 'Melbourne', 'Richmond']

datetimes = []
dates, times = df['Date'], df['KickOff']

for i in range(len(dates)):
    date_str = str(dates[i]) + " " + str(times[i]) + ":00"
    date = dt.datetime.strptime(date_str, "%d-%b-%y %H:%M:%S")
    
    datetimes.append(date)

print(datetimes[-1])

df['Datetime'] = datetimes

df = df.iloc[:, range(2,14)]


df = df.set_index('Datetime')

df_17, df_18, df_16 = df['2017'], df['2018'], df['2016']





2009-06-19 19:40:00


In [5]:
len(df_18['AwayScore'])

45

In [6]:
scores_for, scores_against, num_games = dict(), dict(), dict()

for team in teams:
    scores_for[team], scores_against[team] = [0,0], [0,0]    
    num_games[team] = 0
    
for home_team, away_team, home_g, home_b, away_g, away_b in zip(df_17['HomeTeam'], df_17['AwayTeam'], df_17['HomeGoals'], df_17['HomeBehinds'], df_17['AwayGoals'], df_17['AwayBehinds']):
    scores_for[home_team][0] += home_g
    scores_for[away_team][0] += away_g
    scores_against[home_team][0] += away_g
    scores_against[away_team][0] += home_g
    
    scores_for[home_team][1] += home_b
    scores_for[away_team][1] += away_b
    scores_against[home_team][1] += away_b
    scores_against[away_team][1] += home_b 

    num_games[home_team] += 1
    num_games[away_team] += 1
    
    
    

In [7]:


team_df = pd.DataFrame()

team_df['GamesPlayed'] = pd.Series(num_games)
team_df['For'] = pd.Series(scores_for)
team_df['Against'] = pd.Series(scores_against)


team_df.head()

,GamesPlayed,For,Against
Adelaide,25,"[395, 325]","[292, 255]"
Brisbane,22,"[276, 221]","[367, 324]"
Carlton,22,"[232, 202]","[299, 244]"
Collingwood,22,"[278, 276]","[287, 241]"
Essendon,23,"[319, 277]","[307, 283]"


In [8]:
avgs_for, avgs_against = dict(), dict()
for team in teams:
    avgs_for[team], avgs_against[team] = [0,0], [0,0]
    
for team, games, F, A in zip(team_df.index, team_df['GamesPlayed'], team_df['For'], team_df['Against']) :
    avgs_for[team][0] = F[0]/games
    avgs_for[team][1] = F[1]/games
    avgs_against[team][0] = A[0]/games
    avgs_against[team][1] = A[1]/games

    
team_df['AvgFor'] = pd.Series(avgs_for)
team_df['AvgAgainst'] = pd.Series(avgs_against)


,GamesPlayed,For,Against,AvgFor,AvgAgainst
Adelaide,25,"[395, 325]","[292, 255]","[15.8, 13.0]","[11.68, 10.2]"
Brisbane,22,"[276, 221]","[367, 324]","[12.545454545454545, 10.045454545454545]","[16.681818181818183, 14.727272727272727]"
Carlton,22,"[232, 202]","[299, 244]","[10.545454545454545, 9.181818181818182]","[13.590909090909092, 11.090909090909092]"
Collingwood,22,"[278, 276]","[287, 241]","[12.636363636363637, 12.545454545454545]","[13.045454545454545, 10.954545454545455]"
Essendon,23,"[319, 277]","[307, 283]","[13.869565217391305, 12.043478260869565]","[13.347826086956522, 12.304347826086957]"


In [11]:
team_df

,GamesPlayed,For,Against,AvgFor,AvgAgainst
Adelaide,25,"[395, 325]","[292, 255]","[15.8, 13.0]","[11.68, 10.2]"
Brisbane,22,"[276, 221]","[367, 324]","[12.545454545454545, 10.045454545454545]","[16.681818181818183, 14.727272727272727]"
Carlton,22,"[232, 202]","[299, 244]","[10.545454545454545, 9.181818181818182]","[13.590909090909092, 11.090909090909092]"
Collingwood,22,"[278, 276]","[287, 241]","[12.636363636363637, 12.545454545454545]","[13.045454545454545, 10.954545454545455]"
Essendon,23,"[319, 277]","[307, 283]","[13.869565217391305, 12.043478260869565]","[13.347826086956522, 12.304347826086957]"
Fremantle,22,"[231, 221]","[317, 258]","[10.5, 10.045454545454545]","[14.409090909090908, 11.727272727272727]"
GWS Giants,25,"[336, 305]","[298, 269]","[13.44, 12.2]","[11.92, 10.76]"
Geelong,25,"[343, 289]","[299, 290]","[13.72, 11.56]","[11.96, 11.6]"
Gold Coast,22,"[257, 214]","[337, 289]","[11.681818181818182, 9.727272727272727]","[15.318181818181818, 13.136363636363637]"
Hawthorn,22,"[273, 226]","[301, 249]","[12.409090909090908, 10.272727272727273]","[13.681818181818182, 11.318181818181818]"


In [18]:
t_goals, t_behinds, t_games = 0, 0, 0

for team, games, F, A in zip(team_df.index, team_df['GamesPlayed'], team_df['For'], team_df['Against']):
    t_goals += F[0]
    t_behinds += F[1]
    t_games += games
                 
avg_goals, avg_behinds = t_goals/t_games, t_behinds/t_games

12.917874396135266
11.495169082125605


In [21]:
attacks = dict()
defences = dict()

for team, avg_for, avg_against in zip(team_df.index, team_df['AvgFor'], team_df['AvgAgainst']):
    attacks[team] = [avg_for[0]/avg_goals, avg_for[1]/avg_behinds]
    defences[team] = [avg_against[0]/avg_goals, avg_against[1]/avg_behinds]

team_df['Attack'] = pd.Series(attacks)
team_df['Defence'] = pd.Series(defences)


team_df

,GamesPlayed,For,Against,AvgFor,AvgAgainst,Attack,Defence
Adelaide,25,"[395, 325]","[292, 255]","[15.8, 13.0]","[11.68, 10.2]","[1.2231114435302917, 1.130909855011557]","[0.9041735228122663, 0.8873292708552215]"
Brisbane,22,"[276, 221]","[367, 324]","[12.545454545454545, 10.045454545454545]","[16.681818181818183, 14.727272727272727]","[0.971170191065479, 0.8738848879634757]","[1.2913748555109812, 1.2811706049781275]"
Carlton,22,"[232, 202]","[299, 244]","[10.545454545454545, 9.181818181818182]","[13.590909090909092, 11.090909090909092]","[0.8163459577072142, 0.7987545129801906]","[1.0521010403209357, 0.9648321839958739]"
Collingwood,22,"[278, 276]","[287, 241]","[12.636363636363637, 12.545454545454545]","[13.045454545454545, 10.954545454545455]","[0.9782076562181274, 1.0913675523887751]","[1.009876249405045, 0.9529694932090393]"
Essendon,23,"[319, 277]","[307, 283]","[13.869565217391305, 12.043478260869565]","[13.347826086956522, 12.304347826086957]","[1.0736724008975318, 1.0476990964488337]","[1.0332834704562452, 1.0703929396932128]"
Fremantle,22,"[231, 221]","[317, 258]","[10.5, 10.045454545454545]","[14.409090909090908, 11.727272727272727]","[0.81282722513089, 0.8738848879634757]","[1.115438226694771, 1.020191407667768]"
GWS Giants,25,"[336, 305]","[298, 269]","[13.44, 12.2]","[11.92, 10.76]","[1.0404188481675392, 1.0613154023954612]","[0.922752430815258, 0.9360453876864887]"
Geelong,25,"[343, 289]","[299, 290]","[13.72, 11.56]","[11.96, 11.6]","[1.0620942408376963, 1.0056398403025846]","[0.9258489154824233, 1.0091195629333893]"
Gold Coast,22,"[257, 214]","[337, 289]","[11.681818181818182, 9.727272727272727]","[15.318181818181818, 13.136363636363637]","[0.9043142721153192, 0.8462052761275286]","[1.1858128782212551, 1.1427725457983915]"
Hawthorn,22,"[273, 226]","[301, 249]","[12.409090909090908, 10.272727272727273]","[13.681818181818182, 11.318181818181818]","[0.9606139933365063, 0.8936560392748667]","[1.059138505473584, 0.9846033353072646]"
